# LangChain v1 快速入门示例

这个 Notebook 演示了如何使用 LangChain v1 创建一个简单的 AI Agent。

## 功能特性

- ✅ 系统提示词定义
- ✅ 工具（Tool）的创建和使用
- ✅ 运行时上下文（Runtime Context）
- ✅ 结构化输出（Structured Output）
- ✅ 对话记忆（Conversational Memory）

## 环境要求

本示例使用阿里云 DashScope 的千问（Qwen）模型，需要设置环境变量：
```bash
export DASHSCOPE_API_KEY="your-dashscope-api-key"
```


## 1. 导入依赖


In [ ]:
import os
from dataclasses import dataclass

from langchain.agents import create_agent
from langchain.tools import ToolRuntime, tool
from langchain_qwq import ChatQwen
from langgraph.checkpoint.memory import InMemorySaver


## 2. 定义系统提示词

系统提示词用于定义 Agent 的角色和行为。


In [ ]:
SYSTEM_PROMPT = """你是一位专业的天气预报员，喜欢使用双关语。

你可以使用两个工具：

- get_weather_for_location: 用于获取特定位置的天气信息
- get_user_location: 用于获取用户的位置

如果用户询问天气，请确保你知道位置。如果从问题中可以判断用户指的是他们所在的位置，
请使用 get_user_location 工具来获取他们的位置。"""

print("✅ 系统提示词已定义")


## 3. 定义运行时上下文

上下文允许工具访问运行时信息，例如用户 ID、会话信息等。


In [ ]:
@dataclass
class Context:
    """自定义运行时上下文模式."""

    user_id: str


print("✅ 上下文模式已定义")


## 4. 创建工具（Tools）

工具是 Agent 可以调用的函数。注意 `get_user_location` 如何使用运行时上下文。


In [ ]:
@tool
def get_weather_for_location(city: str) -> str:
    """获取指定城市的天气信息."""
    return f"{city} 总是阳光明媚！"


@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """根据用户 ID 获取用户位置信息."""
    user_id = runtime.context.user_id
    return "佛罗里达" if user_id == "1" else "旧金山"


print("✅ 工具已创建")
print(f"  - {get_weather_for_location.name}: {get_weather_for_location.description}")
print(f"  - {get_user_location.name}: {get_user_location.description}")


## 5. 定义响应格式

使用结构化输出确保 Agent 的响应符合特定模式。


In [ ]:
@dataclass
class ResponseFormat:
    """Agent 的响应模式."""

    # 带有双关语的回复（必填）
    punny_response: str
    # 如果有的话，包含有趣的天气信息（可选）
    weather_conditions: str | None = None


print("✅ 响应格式已定义")


## 6. 配置语言模型

初始化阿里云 DashScope 的千问（Qwen）3 Next 80B 模型并设置参数。


In [ ]:
model = ChatQwen(
    model="qwen3-next-80b-a3b-instruct",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    temperature=0.5,
    max_tokens=1000,
)

print("✅ 模型已配置")
print(f"  模型: Qwen 3 Next 80B")
print(f"  Base URL: https://dashscope.aliyuncs.com/compatible-mode/v1")
print(f"  Temperature: 0.5")
print(f"  Max Tokens: 1000")


## 7. 设置对话记忆

使用内存检查点来保存对话状态。


In [ ]:
checkpointer = InMemorySaver()

print("✅ 检查点已配置（使用内存存储）")
print("  ⚠️  生产环境建议使用持久化存储")


## 8. 创建 Agent

将所有组件组合在一起创建完整的 Agent。


In [ ]:
agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ResponseFormat,
    checkpointer=checkpointer,
)

print("✅ Agent 创建成功！")


## 9. 运行第一轮对话

询问天气信息。Agent 会：
1. 识别需要获取用户位置
2. 调用 `get_user_location` 工具
3. 调用 `get_weather_for_location` 工具
4. 返回结构化响应


In [ ]:
# thread_id 是对话的唯一标识符
config = {"configurable": {"thread_id": "1"}}

print("🤖 用户: 外面天气怎么样？")
print()

response = agent.invoke(
    {"messages": [{"role": "user", "content": "外面天气怎么样？"}]},
    config=config,
    context=Context(user_id="1"),
)

print("💬 Agent 响应:")
print(f"  回复: {response['structured_response'].punny_response}")
print(f"  天气信息: {response['structured_response'].weather_conditions}")


## 10. 继续对话

使用相同的 `thread_id` 继续对话，Agent 会记住之前的上下文。


In [ ]:
print("🤖 用户: 谢谢你！")
print()

response = agent.invoke(
    {"messages": [{"role": "user", "content": "谢谢你！"}]},
    config=config,
    context=Context(user_id="1"),
)

print("💬 Agent 响应:")
print(f"  回复: {response['structured_response'].punny_response}")
print(f"  天气信息: {response['structured_response'].weather_conditions}")


## 11. 测试不同用户

尝试使用不同的用户 ID，看看位置如何变化。


In [ ]:
# 使用不同的 thread_id 开始新对话
config_user2 = {"configurable": {"thread_id": "2"}}

print("🤖 用户 2: 天气如何？")
print()

response = agent.invoke(
    {"messages": [{"role": "user", "content": "天气如何？"}]},
    config=config_user2,
    context=Context(user_id="2"),  # 不同的用户 ID
)

print("💬 Agent 响应:")
print(f"  回复: {response['structured_response'].punny_response}")
print(f"  天气信息: {response['structured_response'].weather_conditions}")
print()
print("📍 注意: 用户 2 的位置是旧金山，而不是佛罗里达")


## 总结

恭喜！你已经成功创建了一个功能完整的 AI Agent，它具备：

- ✅ **理解上下文**并记住对话
- ✅ **智能使用多个工具**
- ✅ **提供结构化响应**（一致的格式）
- ✅ **处理用户特定信息**（通过上下文）
- ✅ **维护对话状态**（跨交互）

## 下一步

你可以基于这个示例：
1. 创建自己的工具函数
2. 自定义系统提示词
3. 定义自己的响应格式
4. 集成真实的 API 和数据源
5. 为 12345 政务服务热线定制 Agent
